## Example for force calculation in a molecule

### Importing modules 

In [1]:
%config IPCompleter.greedy=True
import numpy as np
import os
import bempp.api
from tools.mesh_converter import *
from tools.force_calculation import *
from tools.apbs_tools import *
import griddata

### PQR file and mesh 

In [2]:
grid, q, x_q = pqrtomesh(directory='C:\\Users\\ian\Desktop\\forces_calculation',protein='arg',forcefield='amber',density=0.2,probe_radius=1.4,build_mesh='no')
bempp.api.PLOT_BACKEND = "gmsh" #gmsh o paraview
grid.plot()

### Right-hand side

With that, we can compute the potential due to the charges on the boundary, required for the right-hand side

In [3]:
# Function to calculate the potential by charges at boundary
ep_in = 4.
ep_ex = 80.
k = 0.125

dirichl_space = bempp.api.function_space(grid, "DP", 0)
neumann_space = bempp.api.function_space(grid, "DP", 0)

@bempp.api.real_callable
def charges_fun(x, n, domain_index, result):
    global q, x_q, ep_in
    suma = 0
    for k in range(len(q)):
        suma = suma + q[k]/(np.linalg.norm(x-x_q[k]))
    result[:] = suma/(4*np.pi*ep_in)
    
charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=charges_fun)

rhs = np.concatenate([charged_grid_fun.coefficients, 
                      np.zeros(neumann_space.global_dof_count)])


C:\Users\ian\Anaconda3\envs\bempp\lib\site-packages\scipy\sparse\linalg\dsolve\linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


### Operators and Matrix

Next, we generate the $2\times 2$ block matrix with the single and double layer operators of the Laplace and Yukawa kernels 

In [4]:
# Define Operators
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k)
dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k)

# Matrix Assembly
blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_in/ep_ex)*slp_out
op_discrete = blocked.strong_form()

### Solver

We now use `gmres` from `scipy` to solve the system. 

In [5]:
import inspect
from scipy.sparse.linalg import gmres

array_it, array_frame, it_count = np.array([]), np.array([]), 0
def iteration_counter(x):
        global array_it, array_frame, it_count
        it_count += 1
        frame = inspect.currentframe().f_back
        array_it = np.append(array_it, it_count)
        array_frame = np.append(array_frame, frame.f_locals["resid"])
        #print "It: {0} Error {1:.2E}".format(it_count, frame.f_locals["resid"])        

x, info = gmres(op_discrete, rhs, callback=iteration_counter, tol=1e-8, maxiter=10000, restart = 20000)

print("The linear system was solved in {0} iterations".format(it_count))

The linear system was solved in 161 iterations


The two following `GridFunction` calls store the calculated boundary potential data (separated by $\phi$ and $\frac{\partial \phi}{\partial n}$) for visualization purposes. 

In [6]:
solution_dirichl = bempp.api.GridFunction(dirichl_space, 
                                          coefficients=x[:dirichl_space.global_dof_count])
solution_neumann = bempp.api.GridFunction(neumann_space, 
                                          coefficients=x[dirichl_space.global_dof_count:])
#solution_dirichl.plot()
#solution_neumann.plot()

### Energy and Force calculaton

To compute $\phi_\text{reac}$ at the atoms locations, we use `operators.potential` to then multiply by the charge and add to compute $\Delta G_\text{solv}$. The `332.064` term is just a unit conversion constant.

The forces are calculated from the `force_calculation` module. We separate three components: Fixed charge (qE), dielectric boundary and ionic boundary

In [10]:
slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_q*solution_neumann - dlp_q*solution_dirichl

f_qf,_,_ = fixed_charge_forces(solution_neumann,solution_dirichl,neumann_space,dirichl_space,x_q,q,h=0.001)
f_db,f_ib = boundary_forces(solution_neumann,solution_dirichl,grid,k,ep_ex,ep_in)
f_solv = f_qf + f_db + f_ib

print("Total solvation forces {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(f_solv[0],f_solv[1],f_solv[2]))  
print("Total reaction force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(f_qf[0],f_qf[1],f_qf[2]))
print("Total dielectric boundary force: {:10.4f}{:10.4f}{:10.4f} [kJ/molA]".format(f_db[0],f_db[1],f_db[2]))
print("Total ionic boundary force: {:5.3e} {:5.3e} {:5.3e} [kJ/molA]".format(f_ib[0],f_ib[1],f_ib[2]))

# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 4.184*2*np.pi*332.064*np.sum(q*phi_q).real
print("Total solvation energy: {:7.2f} [kJ/Mol]".format(total_energy))

Total solvation forces    -0.9428   -0.9130   -1.7530 [kJ/molA]
Total reaction force:    34.3522  -10.9618   11.2461 [kJ/molA]
Total dielectric boundary force:   -35.2799   10.0504  -12.9854 [kJ/molA]
Total ionic boundary force: -1.505e-02 -1.689e-03 -1.373e-02 [kJ/molA]
Total solvation energy: -106.21 [kJ/Mol]


## References
[1] Yoon, B. J., & Lenhoff, A. M. (1990). A boundary element method for molecular electrostatics with electrolyte effects. *Journal of Computational Chemistry*, 11(9), 1080-1086.

[2] Dolinsky, T. J., Nielsen, J. E., McCammon, J. A., & Baker, N. A. (2004). PDB2PQR: an automated pipeline for the setup of Poisson–Boltzmann electrostatics calculations. *Nucleic acids research*, 32(suppl_2), W665-W667.

[3] Sanner, M. F., Olson, A. J., & Spehner, J. C. (1995, September). Fast and robust computation of molecular surfaces. In *Proceedings of the eleventh annual symposium on Computational geometry* (pp. 406-407). ACM.

[4] Sergio Decherchi and Walter Rocchia. A general and Robust Ray-Casting-Based
Algorithm for Triangulating Surfaces at the Nanoscale. PLOS ONE, 8(4):1–15, 2013.

[5] Timo Betcke and Matthew W. Scroggs. Bempp-cl: A fast python based just-in-time
compiling boundary element library. Journal of Open Source Software, 6(59):2879,
2021.

[6] S. Decherchi, A. Spitaleri, J. Stone and W. Rocchia, "NanoShaper-VMD interface: computing and visualizing surfaces, pockets and channels in
molecular systems". Bioinformatics, in press, 2018.